In [60]:
import transcriptic 
from transcriptic import commands, Container
from transcriptic.config import Connection
from transcriptic.jupyter import objects
import json
from pysd2cat.data import tx_fcs
from pysd2cat.data import pipeline
from pysd2cat.analysis import biofab_live_dead_analysis as blda

from os.path import expanduser
import requests
import os
import zipfile
import shutil
import fnmatch
import glob
import pandas as pd
import ast

%load_ext autoreload
%reload_ext autoreload
%autoreload 2



run_id='r1dd37mcxv5pf4'
work_dir='data/transcriptic'
Connection.from_file("~/.transcriptic")
tx_config = json.load(open(os.path.join(expanduser("~"), ".transcriptic")))
transcriptic_email = tx_config['email']
transcriptic_token = tx_config['token']



if not os.path.exists(work_dir):
    os.makedirs(work_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [11]:
def get_flow_plate_properties(run_id, transcriptic_email, transcriptic_token, work_dir, download_data=False):
    file_info = tx_fcs.create_fcs_manifest_and_get_files(run_id, 
                                                         transcriptic_email, 
                                                         transcriptic_token,
                                                         fcs_path=work_dir, 
                                                         download_zip=False,
                                                         container_name="Flow (Sytox) Plate 1"
                                                         )
    #Flow plate didn't have the properties, so need to get them from another plate
    inoc_plate_info = tx_fcs.get_plate_well_properties(tx_fcs.get_tx_run(run_id), container_name="Inoculation Plate")
    for well_id, info in file_info.items():
        info['properties'] = inoc_plate_info[well_id.upper()]
    return file_info

In [12]:
plate_properties = get_flow_plate_properties(run_id, transcriptic_email, transcriptic_token, work_dir, download_data=False)

Attempting to fetch 7 datasets...


In [ ]:
def make_row(well, info):
    row = { 
        "well" : well,
        "file" : info['file'],
        "checksum" : info['checksum']
        }
    # Properties don't include sytox, so add manually
    def well_stain(well):
        if 'a' in well or 'b' in well or 'c' in well or 'd' in well:
            return "SYTOX Red Stain"
        else:
            return None
        
    row['stain'] = well_stain(well) 
    
    def expand_options(v):
        print(v)
        kv_pairs = {}
        for option, value in v.items():

In [56]:


def make_row(well, info):
    row = { 
        "well" : well,
        "filename" : info['file'],
        "checksum" : info['checksum']
        }
    # Properties don't include sytox, so add manually
    def well_stain(well):
        if 'a' in well or 'b' in well or 'c' in well or 'd' in well:
            return "SYTOX Red Stain"
        else:
            return None
        
    row['stain'] = well_stain(well) 
    
    def expand_options(v):
        #print(v)
        kv_pairs = {}
        for option, value in v.items():
            if option == 'Reagents':
                #print(option)
                for reagent, properties in value.items():
                    if reagent == 'Ethanol':
                        kv_pairs['kill'] = reagent
                        kv_pairs['kill_volume'] = properties['final_concentration']['qty']
        return kv_pairs
    
    for k, v in info['properties'].items():
        if k == 'Options':
            row.update(expand_options(ast.literal_eval(v)))
        else:
            row[k] = str(v)
    return row

def make_meta_dataframe(run_id, plate_properties):
    data_df = pd.DataFrame()
    for well, info in plate_properties.items():
        row = make_row(well, info)
        data_df = data_df.append(row, ignore_index=True)
        
    data_df['experiment_id'] = run_id
    return data_df
        
meta_df = make_meta_dataframe(run_id, plate_properties)
meta_df

,Container Type,Control,Culture_Volume,Media,Replicate,Storage (C),Strain,checksum,filename,kill,kill_volume,plate_id,source,stain,well,experiment_id
0,96-pcr,{},"{'qty': 1000.0, 'units': 'uL'}","{'YPAD': {'item_id': 378596, 'item_concentrati...",3/8,cold_80,"{'S288c_a': {'item_id': 389331, 'item_concentr...",afe43f51e8ab4acf2274d2a339c877c50942ddd0,data/transcriptic/fcs/CellDeath-EtOH-07-25-201...,Ethanol,29.0,394511,[{'id': 393948}],SYTOX Red Stain,c8,r1dd37mcxv5pf4
1,96-pcr,{},"{'qty': 1000.0, 'units': 'uL'}","{'YPAD': {'item_id': 378596, 'item_concentrati...",1/8,cold_80,"{'S288c_a': {'item_id': 389331, 'item_concentr...",081314b58568f60ffe805f7d75bc680e2873f94f,data/transcriptic/fcs/CellDeath-EtOH-07-25-201...,Ethanol,980.0,394511,[{'id': 393923}],SYTOX Red Stain,a1,r1dd37mcxv5pf4
2,96-pcr,{},"{'qty': 1000.0, 'units': 'uL'}","{'YPAD': {'item_id': 378596, 'item_concentrati...",3/8,cold_80,"{'S288c_a': {'item_id': 389331, 'item_concentr...",d8dba1d96a606cf0605d4ab02ca33ce75c929605,data/transcriptic/fcs/CellDeath-EtOH-07-25-201...,Ethanol,0.0,394511,[{'id': 393949}],SYTOX Red Stain,c9,r1dd37mcxv5pf4
3,96-pcr,{},"{'qty': 1000.0, 'units': 'uL'}","{'YPAD': {'item_id': 378596, 'item_concentrati...",1/8,cold_80,"{'S288c_a': {'item_id': 389331, 'item_concentr...",5d5805371b237efff255eef909da8052589c6b0c,data/transcriptic/fcs/CellDeath-EtOH-07-25-201...,Ethanol,370.0,394511,[{'id': 393925}],SYTOX Red Stain,a3,r1dd37mcxv5pf4
4,96-pcr,{},"{'qty': 1000.0, 'units': 'uL'}","{'YPAD': {'item_id': 378596, 'item_concentrati...",1/8,cold_80,"{'S288c_a': {'item_id': 389331, 'item_concentr...",a1deb04c2d208f29c6764b26fc0fe8eea87d2dc7,data/transcriptic/fcs/CellDeath-EtOH-07-25-201...,Ethanol,570.0,394511,[{'id': 393924}],SYTOX Red Stain,a2,r1dd37mcxv5pf4
5,96-pcr,{},"{'qty': 1000.0, 'units': 'uL'}","{'YPAD': {'item_id': 378596, 'item_concentrati...",1/8,cold_80,"{'S288c_a': {'item_id': 389331, 'item_concentr...",e11d90db2a7e91499b10b7b998b88071ea2f1314,data/transcriptic/fcs/CellDeath-EtOH-07-25-201...,Ethanol,105.0,394511,[{'id': 393928}],SYTOX Red Stain,a6,r1dd37mcxv5pf4
6,96-pcr,{},"{'qty': 1000.0, 'units': 'uL'}","{'YPAD': {'item_id': 378596, 'item_concentrati...",1/8,cold_80,"{'S288c_a': {'item_id': 389331, 'item_concentr...",01aca22eff88dfe90cc7278e6eaed3d5f1f810f0,data/transcriptic/fcs/CellDeath-EtOH-07-25-201...,Ethanol,64.0,394511,[{'id': 393929}],SYTOX Red Stain,a7,r1dd37mcxv5pf4
7,96-pcr,{},"{'qty': 1000.0, 'units': 'uL'}","{'YPAD': {'item_id': 378596, 'item_concentrati...",1/8,cold_80,"{'S288c_a': {'item_id': 389331, 'item_concentr...",ea6a2b4e3c5b20eddb56cf6242a0e89117290831,data/transcriptic/fcs/CellDeath-EtOH-07-25-201...,Ethanol,170.0,394511,[{'id': 393927}],SYTOX Red Stain,a5,r1dd37mcxv5pf4
8,96-pcr,{},"{'qty': 1000.0, 'units': 'uL'}","{'YPAD': {'item_id': 378596, 'item_concentrati...",2/8,cold_80,"{'S288c_a': {'item_id': 389331, 'item_concentr...",6c6844a3f17a3fba1cad6b194b2b844bf75d6118,data/transcriptic/fcs/CellDeath-EtOH-07-25-201...,Ethanol,0.0,394511,[{'id': 393940}],SYTOX Red Stain,b9,r1dd37mcxv5pf4
9,96-pcr,{},"{'qty': 1000.0, 'units': 'uL'}","{'YPAD': {'item_id': 378596, 'item_concentrati...",2/8,cold_80,"{'S288c_a': {'item_id': 389331, 'item_concentr...",9b18418dd47914f6bbb22d1331fd87f014a2f001,data/transcriptic/fcs/CellDeath-EtOH-07-25-201...,Ethanol,29.0,394511,[{'id': 393939}],SYTOX Red Stain,b8,r1dd37mcxv5pf4


In [66]:
data_dir='.'
df = pipeline.get_data_and_metadata_df(meta_df, data_dir, fraction=None, max_records=30000)

record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_C8.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_C8.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_A1.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_A1.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_C9.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_C9.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_A3.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_A3.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_A2.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Pla

record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_G9.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_G9.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_E1.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_E1.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_D5.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_D5.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_B12.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_B12.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_D4.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-P

record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_C7.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_C7.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_B3.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_B3.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_E12.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_E12.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_B2.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_B2.fcs
record[filename]: data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-Plate_C6.fcs
.
data/transcriptic/fcs/CellDeath-EtOH-07-25-2019-Run1_Sytox-YeastGates_SytoxYG-P

In [67]:
df.stain.unique()

array(['SYTOX Red Stain', None], dtype=object)

In [64]:

blda.train_models_for_stats(df, 
                            data_dir=work_dir, 
                            fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'])


{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain'}
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 980.0, 'stain': None}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


HI
----------------------------------------------------------------------------------------------------
Starting run at time 22:31:18
Normalizing training and testing splits...
Starting Classification training...
Training time was: 40.89 seconds
Testing time was: 0.64 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                       Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  QgDA8YOg58WPP  2019-07-28  22:31:18  random_forest_classification  Hamed        2           0.994     0.994              1.0        1.0                0.994     0.993     

...

HI
----------------------------------------------------------------------------------------------------
Starting run at time 23:35:41
Normalizing training and testing splits...
Starting Classification training...
Training time was: 62.62 seconds
Testing time was: 1.57 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                 Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  5ZyJ9Xak7e2xw  2019-07-28  23:35:41  random_forest_classification  Hamed        2           0.647     0.647              0.709      0.677              0.673     

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


HI
----------------------------------------------------------------------------------------------------
Starting run at time 23:36:53
Normalizing training and testing splits...
Starting Classification training...
Training time was: 52.32 seconds
Testing time was: 1.77 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                    Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  52LjE9Q3LRJMM  2019-07-28  23:36:53  random_forest_classification  Hamed        2           0.638     0.638              0.691      0.673              0.64      0.637      0.

In [105]:
pred_df = blda.train_models_for_prediction(df, data_dir=work_dir,
                                 fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'],
                                           combine_stains=False,
                                          overwrite=True,
                                          output_col = 'live')
pred_df.to_csv(os.path.join(work_dir, 'data.csv'))

SYTOX Red Stain
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain', 'prediction': True}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W', 'index']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']

----------------------------------------------------------------------------------------------------
Starting run at time 11:39:45
Normalizing training and testing splits...
Starting Classification training...
Training time was: 29.51 seconds
Testing time was: 0.66 seconds
Prediction time of untested data was: 12.722727298736572

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                      Data and Split Description Normalized Num Features Normalized Feature Extraction Was Unt

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']

----------------------------------------------------------------------------------------------------
Starting run at time 11:40:47
Normalizing training and testing splits...
Starting Classification training...
Training time was: 35.19 seconds
Testing time was: 0.65 seconds
Prediction time of untested data was: 14.104676961898804

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                         Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Pr

In [106]:
# Train on just side scatter
pred_df = blda.train_models_for_prediction(df, data_dir=work_dir,
                                 fcs_columns=[ 'SSC-A','SSC-H', 'SSC-W'],
                                           combine_stains=False,
                                          overwrite=True,
                                          additional_description={
                                              'channels' : [ 'SSC-A','SSC-H', 'SSC-W']
                                          },
                                          output_col = 'live_ss')
pred_df.to_csv(os.path.join(work_dir, 'data.csv'))

SYTOX Red Stain
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain', 'prediction': True, 'channels': ['SSC-A', 'SSC-H', 'SSC-W']}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W', 'index']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['SSC-A', 'SSC-H', 'SSC-W']

----------------------------------------------------------------------------------------------------
Starting run at time 11:46:21
Normalizing training and testing splits...
Starting Classification training...
Training time was: 28.02 seconds
Testing time was: 0.83 seconds
Prediction time of untested data was: 18.304264783859253

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                                                               Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  a3Bb16mloEPbo  2

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['SSC-A', 'SSC-H', 'SSC-W']

----------------------------------------------------------------------------------------------------
Starting run at time 11:47:26
Normalizing training and testing splits...
Starting Classification training...
Training time was: 25.12 seconds
Testing time was: 0.83 seconds
Prediction time of untested data was: 20.649171113967896

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                                                  Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  EMv1AWBpzQr7x  2019-07-30  11

In [107]:
pred_df.columns

Index(['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W', 'index', 'live_ss'], dtype='object')

In [98]:
len(pred_df.loc[pred_df.stain == 'SYTOX Red Stain'])

1098660